In [1]:
import sys
import numpy as np

sys.path.insert(0, "..")
from src.indexers import IndexerFlat, IndexerPQ, IndexerIVFPQ, DistanceComputationModes

%load_ext autoreload
%autoreload 2

### setup data and hyperparameters

In [2]:
n = 10000  # number of points
D = 8  # hidden dim
n_train = 1000  # number of points to fir kmeans
random_state = 42  # random state to choose train sample
d = 2  # block dim
assert D % d == 0
nb = D // d  # number of blocks
nc = 256  # number of codes, <= 2**8
nl = 10  # number of leaves, <= n
m = 3  # number of queries
k = 10  # number of closest points to retrieve

np.random.seed(0)
data = np.random.uniform(size=(n, D))
queries = np.random.uniform(size=(m, D))

### flat

In [3]:
%%time
flat = IndexerFlat()
flat.fit(data)

CPU times: user 29 µs, sys: 27 µs, total: 56 µs
Wall time: 66 µs


In [4]:
%%time
res_flat = flat.search(queries, k=k)

CPU times: user 7.48 ms, sys: 0 ns, total: 7.48 ms
Wall time: 6.93 ms


In [5]:
res_flat

SearchResult(indices=array([[2610, 6939, 7066, 6540, 6132, 5488, 2995, 8057, 8110, 9541],
       [7675, 4883, 9183, 9362, 7864, 2017, 1890, 3246, 9448, 6925],
       [4713, 5222, 2356, 7372, 7804,  137, 1533,  554, 8186, 7431]]), distances=array([[0.21853444, 0.26512856, 0.34076707, 0.36802012, 0.37083441,
        0.38281394, 0.3907406 , 0.39185182, 0.40083054, 0.40343069],
       [0.20758142, 0.28236194, 0.31947943, 0.32950834, 0.37668377,
        0.3839569 , 0.41757126, 0.42343857, 0.42984467, 0.43013169],
       [0.27695859, 0.31096122, 0.38545311, 0.38818866, 0.39941996,
        0.42291079, 0.43586913, 0.45110217, 0.46084486, 0.461539  ]]))

In [6]:
%timeit flat.search(queries, k=k)

2.13 ms ± 28.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* a way to compare approximate nn with exact nn

In [7]:
from sklearn.metrics import ndcg_score
from src.indexers import SearchResult, cdist_nd

def ndcg(res: SearchResult, k=None) -> float:
    y_true = cdist_nd(queries[:, None, :], data[res.indices])  # [m, 1, D], [m, k, D] -> [m, 1, k]
    return ndcg_score(1 / y_true.squeeze(1), 1 / res.distances, k=k)

### pq

In [8]:
%%time
pq = IndexerPQ(D=D, d=d, nc=nc)
pq.fit(data, n_train=n_train, random_state=random_state)

100%|██████████| 4/4 [00:11<00:00,  2.89s/it]

CPU times: user 40 s, sys: 1min 35s, total: 2min 15s
Wall time: 11.6 s


In [9]:
%%time
res_pq_adc = pq.search(queries, k=k, dc_mode=DistanceComputationModes.ADC)

CPU times: user 19.6 ms, sys: 36 ms, total: 55.6 ms
Wall time: 5.11 ms


In [10]:
ndcg(res_pq_adc)

0.9987412339770075

In [11]:
%%time
res_pq_sdc = pq.search(queries, k=k, dc_mode=DistanceComputationModes.SDC)

CPU times: user 8.09 ms, sys: 199 µs, total: 8.29 ms
Wall time: 7.61 ms


In [12]:
ndcg(res_pq_sdc)

0.9979474072477782

In [13]:
pq.n_loops = 2
res_pq_2 = pq.search(queries, k=k)
ndcg(res_pq_2)

0.9987412339770075

In [14]:
pq.n_loops = 3
res_pq_3 = pq.search(queries, k=k)
ndcg(res_pq_3)

0.9987412339770075

In [15]:
for n_loops in [0,2,3]:
    print(n_loops)
    pq.n_loops = n_loops
    %timeit pq.search(queries, k=k)
    print("=" * 30)

0
3.5 ms ± 52.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2
127 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
3
65.7 ms ± 606 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
pq.n_loops = 0
%timeit pq.search(queries, k=k, dc_mode=DistanceComputationModes.SDC)

3.15 ms ± 41 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### ivfpq

In [17]:
%%time
ivfpq = IndexerIVFPQ(D=D, d=d, nc=nc, num_leaves=nl)
ivfpq.fit(data, n_train=n_train, random_state=random_state)

100%|██████████| 4/4 [03:44<00:00, 56.22s/it]

CPU times: user 10min 51s, sys: 30min 12s, total: 41min 3s
Wall time: 3min 45s


In [18]:
%time
res_ivfpq_1 = ivfpq.search(queries, k=k, nprobe=1)

CPU times: user 6 µs, sys: 17 µs, total: 23 µs
Wall time: 5.48 µs


In [19]:
ndcg(res_ivfpq_1)

0.9971601991732161

In [20]:
%time
res_ivfpq_5 = ivfpq.search(queries, k=k, nprobe=5)

CPU times: user 0 ns, sys: 1 µs, total: 1 µs
Wall time: 4.77 µs


In [21]:
ndcg(res_ivfpq_5)

0.9984798822454827

In [22]:
%timeit ivfpq.search(queries, k=k, nprobe=1)

2.45 ms ± 17 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit ivfpq.search(queries, k=k, nprobe=5)

11.6 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
